In [ ]:
!pip install --upgrade -q accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, pipeline, TextStreamer)

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = 'unsloth/Meta-Llama-3.1-8B',
                                                   quantization_config = BitsAndBytesConfig(load_in_8bit = True,
                                                                                            max_seq_length = 2048,
                                                                      bnb_4bit_compute_dtype = getattr(torch, 'float16'),
                                                                      torch_dtype=torch.float16))
llama_model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = 'unsloth/Meta-Llama-3.1-8B',
                                                max_seq_length = 2048,
                                                trust_remote_code = True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}


###Output:
{}"""

EOS_Token = llama_tokenizer.eos_token

def formatting(dataset):
  instructions = dataset['instruction']
  inputs = dataset['input']
  outputs = dataset['output']
  texts = []
  for instruction , input, output in zip(instructions, inputs, outputs):
    text = format.format(instruction, input, output) + EOS_Token
    texts.append(text)
  return {'text':texts}

pass
from datasets import load_dataset
dataset = load_dataset('csv', data_files='/content/AgDA_with_instructions.csv', split='train')
dataset = dataset.map(formatting, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3667 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model = llama_model,
    train_dataset = dataset,
    peft_config = LoraConfig(
       r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_rslora = False,
    loftq_config = None,
    ),
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 29,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_bnb_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        fp16=True,
        gradient_checkpointing=True,
    )
)

Adding EOS to train dataset:   0%|          | 0/3667 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3667 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3667 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

In [ ]:
!pip install fastapi uvicorn

In [ ]:
!pip install pyngrok

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import LlamaForCausalLM, PreTrainedTokenizerFast

nest_asyncio.apply()

app = FastAPI()

class InputData(BaseModel):
    input_text: str

@app.post("/predict")
async def generate_text(data: InputData):
    try:
        instruction = "Generate text based on the input. Be humane like, always expantiate on your answer, be very conversative. After every successful answer, always ask if there is anything needed again, just be friendly and also very conversative."

        formatted_input = format.format(instruction, data.input_text, "")

        inputs = llama_tokenizer(formatted_input, return_tensors="pt").to("cuda")

        outputs = llama_model.generate(**inputs, max_length=512, use_cache=False)
        output_text = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

        return {"generated_text": output_text}

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



from pyngrok import ngrok

ngrok.set_auth_token("2lF5wXiCnwiL7Gw02aZYf670KtN_6cgH5KX53byFqAiJbH7e6")
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)